In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [5]:
train = pd.read_csv('train.tsv', sep='\t',index_col=0)
test = pd.read_csv('test.tsv', sep='\t',index_col=0)
sample = pd.read_csv('sample_submit.csv', index_col=0, header=None)

In [32]:
train['left'] = train['left_weight'] * train['left_distance']
train['right'] = train['right_weight'] * train['right_distance']
train['per'] = train['left'] / train['right']
test['left'] = test['left_weight'] * test['left_distance']
test['right'] = test['right_weight'] * test['right_distance']
test['per'] = test['left'] / test['right']

In [33]:
train.columns

Index(['class', 'left_weight', 'left_distance', 'right_weight',
       'right_distance', 'left', 'right', 'per'],
      dtype='object')

In [34]:
use_columns = ['per']
train_y = train['class']
train_X = train[use_columns]
test_X = test[use_columns]

In [35]:
train_X

,per
2,0.666667
9,1.333333
11,2.000000
13,0.600000
16,1.200000
...,...
611,0.120000
612,3.125000
614,2.500000
616,5.000000


In [42]:
scores = []

# KFoldを用いてクロスバリデーションの分割を行う
kf = KFold(n_splits=5)
for tr_idx, va_idx in kf.split(train_X):
    tr_X, va_X = train_X.iloc[tr_idx], train_X.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # 学習の実行、バリデーションデータの予測値の出力、スコアの計算
    model = xgb.XGBClassifier(n_estimators=100, max_depth=2, learning_rate=0.1)
    model.fit(tr_X, tr_y)
    va_pred = model.predict(va_X)
    score = accuracy_score(va_y, va_pred)
    scores.append(score)

# スコアの表示
print(scores)
print(np.mean(scores))

[1.0, 1.0, 1.0, 1.0, 1.0]
1.0


In [43]:
# 提出予測値の生成
pred = model.predict(test_X)

In [44]:
submit = sample.copy()
submit[1] = pred

In [45]:
submit.head()

,1
0,
0,0
1,0
3,2
4,0
5,1


In [46]:
submit.to_csv('submit.csv', header=None)

評価結果：0.9936102236421726